In [ ]:
!pip install -U albumentations
!pip install torchmetrics

Для оубчения модели необхожимо в словаре train_config указать нужные параметры для обучения, а именно:
* путь для валидационного и обучаещего датасета - train_data_path, valid_data_path соответсвенно.<br>
* is_custom_model - является ли модель для обучения собственной архитектурой (True/False)
* img_size - размер фотографии
* num_epochs - количество эпох
* batch_size - размер батчаей
* lr - learning_rate для градиентного спуска

В методе train_model класса Model_train можно указать нужную функцию потерь и тип градиентного спуска, заменив значения переменных <br>
criterion = nn.CrossEntropyLoss()<br>
optimizer = optim.Adam(self.model.parameters(), lr=self.train_config['lr'])<br>
на нужные<br>

в train_albumentations_transforms можно заменит параметры аугментации на нужные.
Для запуск тестирования модели необходимо в словаре test_config указать нужные параметры. Описание параметров совпадает с параметрами для train_config.

Более подробные описания методов представленна в коде в комментариях.

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# Классы для обучения, загрузки датасета и класс модели

In [ ]:
# Класс загрузчика данных. Принимает путь до датсета, размер фотографии и экземпляр
# класса для аугментации
class Data_loader:
    def __init__(self, dataset_path, img_size, alb_transform=None):
        self.dataset = ImageFolder(root=dataset_path)
        self.img_size = img_size
        self.alb_transform = alb_transform
        self.to_tensor_transform = transforms.ToTensor()

    def __len__(self):
        return len(self.dataset)


    def __getitem__(self, idx):
        img, target = self.dataset[idx]
        img = img.convert("RGB")

        if self.alb_transform:
            img = self.alb_transform(image=np.array(img))['image']
            img = self.to_tensor_transform(img)

            return img, target


    def get_test_item(self, idx):
        test_transform = transforms.Resize((self.img_size, self.img_size))
        img = self.to_tensor_transform(img)
        img, target = self.test_dataset[idx]
        img = img.convert("RGB")
        img = test_transform(img)
        img = self.to_tensor_transform(img)

        return img, target


In [ ]:
# Класс для обучения модели. При иннициализации принимает архитектуру модели, класс загрузчика для тренировочного и валидационного датасета
# И словарь с параметрами обучения
# При необходимоти тут можно поменять функцию потерь и метод градиентного спуска
class Train_model:
    def __init__(self, model, train_dataset, valid_dataset, train_config):
        self.train_dataset = train_dataset
        self.train_dataset = valid_dataset
        self.train_config = train_config
        self.model = model
        self.train_loader = DataLoader(train_dataset, batch_size = self.train_config['batch_size'])
        self.valid_loader = DataLoader(valid_dataset, batch_size = self.train_config['batch_size'])
        self.is_custom_model = train_config['is_custom_model']

        train_targets = []
        for img, lbl in self.train_loader:
            train_targets.append(lbl.numpy())

        valid_targets = []
        for img, lbl in self.valid_loader:
            valid_targets.append(lbl.numpy())

        self.train_targets = np.concatenate(train_targets)
        self.valid_targets = np.concatenate(valid_targets)


    def forward_dataset(self, dataloader):# Прогон всего датасета через модель
        predict_list = []
        with torch.no_grad():
            for i, data in enumerate(dataloader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                predict_list.append(predicted.tolist())


        return np.concatenate(predict_list)


    def train_model(self): # метод тренировки модели
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        criterion = nn.CrossEntropyLoss() # Если класса всего два, то поставить бинарную кросс энтрапию
        optimizer = optim.Adam(self.model.parameters(), lr=self.train_config['lr']) # поменять метод градиентного спуска, если нужно

        if self.is_custom_model:
            self.model.set_custom_weights()

        for epoch in range(self.train_config['num_epochs']):
            print(f"epoch {epoch + 1} / {train_config['num_epochs']}")

            for i, data in enumerate(self.train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()

                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            self.model.eval()

            valid_predict = self.forward_dataset(self.valid_loader)
            train_predict = self.forward_dataset(self.train_loader)

            train_accurcay = round(accuracy_score(self.train_targets, train_predict), 3)
            valid_accuracy = round(accuracy_score(self.valid_targets, valid_predict), 3)
            valid_f1 = round(f1_score(self.valid_targets, valid_predict, average='macro'), 3)
            valid_precision = round(precision_score(self.valid_targets, valid_predict, average='macro'), 3)
            valid_recall = round(recall_score(self.valid_targets, valid_predict, average='macro'), 3)

            self.model.train()

            print(f'Train_loss: {loss.item()}, Train_accuracy: {train_accurcay}, Valid_accuracy: {valid_accuracy}, Valid_f1: {valid_f1}, valid_precision: {valid_precision}, valid_recall: {valid_recall}')

In [ ]:
class CustomCNN(nn.Module): # Настроить для себя архитектуру модели, если нужно
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x

    def set_custom_weights(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.fc2.weight, a=nn.init.calculate_gain('relu'))
        nn.init.zeros_(self.fc2.bias)


# Обучение

## Настройка параметров обучения

In [ ]:
# Выбрать нужные параметры для процесса обучения модели.
train_config = {
    'train_data_path': '/content/drive/MyDrive/hackathon/train',
    'valid_data_path': '/content/drive/MyDrive/hackathon/valid',
    'is_custom_model': False, # Если используется собственная архитектура CustomCNN, поменять на True
    'img_size': 224,
    'num_epochs': 15,
    'batch_size': 50,
    'lr': 0.0001 # learning_rate для градиентного спуска
}

## Настройка аугментации

In [ ]:
# Подорбрать нужные параметры для аугментации. Если нужно посмортеть результат аугментации - вызвать метод train_dataset.__getitem__(idx)
train_albumentations_transforms = A.Compose([
    A.Resize(width=train_config['img_size'], height=train_config['img_size']),
    A.HorizontalFlip(),
    A.RandomBrightnessContrast(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

valid_albumentations_transforms = A.Compose([A.Resize(width=train_config['img_size'], height=train_config['img_size'])])

In [ ]:
train_dataset = Data_loader(dataset_path=train_config['train_data_path'],
                            img_size=train_config['img_size'],
                            alb_transform=train_albumentations_transforms
                            ) # Иннициализация датасета


valid_dataset = Data_loader(dataset_path=train_config['valid_data_path'],
                            img_size=train_config['img_size'],
                            alb_transform=valid_albumentations_transforms
                            )

## Обучение модели

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Указать нужную модель для обучения: cвою - CustomCNN или же из torchvision.models
# model = CustomCNN(num_classes=4).to(device) - пример объявления CustomCNN. num_classes - количество классов
model = models.resnet18(pretrained=True).to(device) # pretrained=True - если нужна предобученная модель




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 113MB/s]


In [ ]:
# Создание экземпляра класса для обучения
trainer = Train_model(
    model = model,
    train_dataset = train_dataset,
    valid_dataset = valid_dataset,
    train_config=train_config,
)

In [ ]:
# Запустить обучение
warnings.filterwarnings('ignore')
trainer.train_model()

epoch 1 / 15
Train_loss: 1.2824363708496094, Train_accuracy: 0.088, Valid_accuracy: 0.072, Valid_f1: 0.014, valid_precision: 0.057, valid_recall: 0.1
epoch 2 / 15
Train_loss: 0.18865278363227844, Train_accuracy: 0.077, Valid_accuracy: 0.071, Valid_f1: 0.013, valid_precision: 0.007, valid_recall: 0.1
epoch 3 / 15
Train_loss: 0.11989251524209976, Train_accuracy: 0.071, Valid_accuracy: 0.071, Valid_f1: 0.013, valid_precision: 0.007, valid_recall: 0.1
epoch 4 / 15
Train_loss: 0.07783243805170059, Train_accuracy: 0.072, Valid_accuracy: 0.071, Valid_f1: 0.013, valid_precision: 0.007, valid_recall: 0.1
epoch 5 / 15
Train_loss: 0.07300681620836258, Train_accuracy: 0.071, Valid_accuracy: 0.071, Valid_f1: 0.013, valid_precision: 0.007, valid_recall: 0.1
epoch 6 / 15
Train_loss: 0.2533874809741974, Train_accuracy: 0.071, Valid_accuracy: 0.071, Valid_f1: 0.013, valid_precision: 0.007, valid_recall: 0.1
epoch 7 / 15
Train_loss: 0.11845263093709946, Train_accuracy: 0.071, Valid_accuracy: 0.071, Vali

In [ ]:
trained_model = trainer.model

# Тестирование

In [ ]:
# Класс для тестирования модель. При иннициализации класса получает обученую модель и словарь c параметрами для тестирования
class Test_model:
    def __init__(self, test_dataset, test_config):
        self.test_dataset = test_dataset
        self.test_config = test_config
        self.model = test_config['model']

        self.test_loader = DataLoader(test_dataset, batch_size = self.test_config['batch_size'])

        test_targets = []
        for img, lbl in self.test_loader:
            test_targets.append(lbl.numpy())

        self.test_targets = np.concatenate(test_targets)


    def test_forward(self):
        predict_list = []
        with torch.no_grad():
            for i, data in enumerate(self.test_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                predict_list.append(predicted.tolist())

        return np.concatenate(predict_list)


## Настройка параметров тесирования

In [ ]:
# Настроить параметры для тестирования
test_config = {
    'test_data_path': '/content/drive/MyDrive/hackathon/test',
    'model': trained_model,
    'img_size': 224, # указать тот же размер изображения, что и был при обучении
    'batch_size': 16 # указать тот же размер батча, что и был при обучении
}

In [ ]:
test_albumentations_transforms = A.Compose([A.Resize(width=test_config['img_size'], height=test_config['img_size'])])

test_dataset = Data_loader(dataset_path=test_config['test_data_path'],
                            img_size=train_config['img_size'],
                            alb_transform=test_albumentations_transforms
                            )

In [ ]:
tester = Test_model(
    test_config=test_config,
    test_dataset = test_dataset
)

In [ ]:
# Запустить тестовый прогон мрдели
test_predict = tester.test_forward() # возвращает предсказание модели
test_targets = tester.test_targets # возвращает true targets тестовой выборки

## Рассчет метрик

In [ ]:
import torch
import torchmetrics
from torchmetrics import MetricTracker, MetricCollection
from torchmetrics import Accuracy, F1Score, Precision, Recall, CohenKappa

num_classes = 10

list_of_metrics = [Accuracy(task="multiclass", num_classes=num_classes),
                   F1Score(task="multiclass", num_classes=num_classes),
                   Precision(task="multiclass",num_classes=num_classes),
                   Recall(task="multiclass",num_classes=num_classes)
                   ] # Указание нужных метрик для рассчета

maximize_list=[True,True,True,True]

metric_coll = MetricCollection(list_of_metrics)
tracker = MetricTracker(metric_coll, maximize=maximize_list)


pred = torch.Tensor(test_predict)

label = torch.Tensor(test_targets)

tracker.increment()
tracker.update(pred, label)

for key, val in tracker.compute_all().items():
    print(key,val)

MulticlassAccuracy tensor([0.0712])
MulticlassF1Score tensor([0.0712])
MulticlassPrecision tensor([0.0712])
MulticlassRecall tensor([0.0712])


# Вывод
По резултатам тестирования модель имеет низкую точность, все метрики ранвы 0.0712. При дальнейшей разработке необходимо будет попробоавть инные параметры для аугментации данных, а также изменить размер батчей при обучении, lr. Попрбовать архитектуру googlenet, так как в датасете содержутся объекты на фотография с маленьки размерами фитч.